# Google ASL challange
Why not sign google assistant 🤙

To do this the image of a hand has to be recognized accordingly


## Imports and Constants

In [1]:
!pip install pytorch-lightning weightwatcher levenshtein

import pathlib

import polars as pl
import numpy as np
import pandas as pd

import pytorch_lightning
import weightwatcher
import Levenshtein

import matplotlib
import plotly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.8/69.8 kB 3.5 MB/s eta 0:00:00


/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [2]:
Basepath = pathlib.Path('/kaggle/input/asl-fingerspelling')
train_CSV = Basepath / 'train.csv'
train_dir = Basepath / 'train_landmarks'

supplemental_landmarks_dir = Basepath / 'supplemental_landmarks'
supplemental_landmarks_CSV = Basepath / 'supplemental_metadata.csv'

char_to_predict_index = Basepath / 'character_to_prediction_index.json'
# list(train_dir.iterdir())

## EDA

In [4]:
train1df = pl.read_parquet(train_dir.iterdir().__next__())
display(train1df)
display(train1df.describe())

frame,x_face_0,x_face_1,x_face_2,x_face_3,x_face_4,x_face_5,x_face_6,x_face_7,x_face_8,x_face_9,x_face_10,x_face_11,x_face_12,x_face_13,x_face_14,x_face_15,x_face_16,x_face_17,x_face_18,x_face_19,x_face_20,x_face_21,x_face_22,x_face_23,x_face_24,x_face_25,x_face_26,x_face_27,x_face_28,x_face_29,x_face_30,x_face_31,x_face_32,x_face_33,x_face_34,x_face_35,…,z_pose_18,z_pose_19,z_pose_20,z_pose_21,z_pose_22,z_pose_23,z_pose_24,z_pose_25,z_pose_26,z_pose_27,z_pose_28,z_pose_29,z_pose_30,z_pose_31,z_pose_32,z_right_hand_0,z_right_hand_1,z_right_hand_2,z_right_hand_3,z_right_hand_4,z_right_hand_5,z_right_hand_6,z_right_hand_7,z_right_hand_8,z_right_hand_9,z_right_hand_10,z_right_hand_11,z_right_hand_12,z_right_hand_13,z_right_hand_14,z_right_hand_15,z_right_hand_16,z_right_hand_17,z_right_hand_18,z_right_hand_19,z_right_hand_20,sequence_id
i16,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,i64
0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,-0.533426,-0.190706,-0.521763,-0.162169,-0.509827,0.056325,-0.055879,0.27666,0.098482,0.749564,0.589746,0.791397,0.645634,0.698711,0.590977,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,260086012
1,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,-0.58812,-0.5262,-0.657358,-0.48561,-0.607399,-0.054204,0.054684,0.523206,0.700682,1.405967,1.514514,1.494215,1.598495,1.431296,1.541956,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,260086012
2,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,…,-0.032193,-0.39854,-0.046164,-0.374162,-0.034443,-0.061847,0.061735,0.516559,0.648936,1.241568,1.334954,1.313005,1.406847,1.261311,1.356719,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,260086012
0,0.803879,0.770967,0.788241,0.767522,0.769132,0.774359,0.792784,0.706465,0.797502,0.795765,0.797372,0.804141,0.805098,0.806497,0.806476,0.805468,0.804829,0.805005,0.805452,0.776552,0.767368,0.667578,0.749977,0.733856,0.71907,0.702759,0.763095,0.719115,0.738271,0.703644,0.69569,0.692406,0.755963,0.702086,0.676335,0.680558,…,-3.15512,-2.333876,-3.114451,-2.174918,-2.887985,-0.163694,0.17184,0.16747,0.692984,1.560448,2.020703,1.6414,2.131315,0.586547,0.977884,0.000001,-0.042928,-0.072611,-0.105344,-0.135671,-0.041546,-0.099427,-0.13735,-0.158361,-0.057472,-0.125935,-0.164229,-0.17842,-0.07881,-0.169536,-0.18174,-0.16183,-0.10233,-0.167616,-0.163522,-0.141645,260186830
1,0.806138,0.777491,0.793601,0.77266,0.775697,0.7804,0.797234,0.706678,0.801277,0.79948,0.800195,0.806531,0.80766,0.809197,0.809226,0.808582,0.808381,0.809062,0.809183,0.782731,0.77248,0.668218,0.753362,0.735854,0.720057,0.703703,0.767332,0.719257,0.740153,0.702682,0.694617,0.69372,0.758932,0.702641,0.676029,0.683188,…,-3.197925,-2.173807,-3.192063,-2.00087,-2.935865,-0.132117,0.140401,0.219664,0.653416,1.637446,1.944947,1.72049,2.052697,0.667451,0.895529,0.000001,-0.044215,-0.073791,-0.106809,-0.136355,-0.038788,-0.091494,-0.124303,-0.141582,-0.054023,-0.118759,-0.154212,-0.166522,-0.074457,-0.166184,-0.178615,-0.157466,-0.096936,-0.165898,-0.163766,-0.142122,260186830
2,0.806748,0.776365,0.792611,0.772168,0.77459,0.779472,0.796899,0.709607,0.801433,0.799877,0.800941,0.807142,0.808244,0.809785,0.809676,0.808854,0.808435,0.808905,0.808301,0.781634,0.77166

describe,frame,x_face_0,x_face_1,x_face_2,x_face_3,x_face_4,x_face_5,x_face_6,x_face_7,x_face_8,x_face_9,x_face_10,x_face_11,x_face_12,x_face_13,x_face_14,x_face_15,x_face_16,x_face_17,x_face_18,x_face_19,x_face_20,x_face_21,x_face_22,x_face_23,x_face_24,x_face_25,x_face_26,x_face_27,x_face_28,x_face_29,x_face_30,x_face_31,x_face_32,x_face_33,x_face_34,…,z_pose_18,z_pose_19,z_pose_20,z_pose_21,z_pose_22,z_pose_23,z_pose_24,z_pose_25,z_pose_26,z_pose_27,z_pose_28,z_pose_29,z_pose_30,z_pose_31,z_pose_32,z_right_hand_0,z_right_hand_1,z_right_hand_2,z_right_hand_3,z_right_hand_4,z_right_hand_5,z_right_hand_6,z_right_hand_7,z_right_hand_8,z_right_hand_9,z_right_hand_10,z_right_hand_11,z_right_hand_12,z_right_hand_13,z_right_hand_14,z_right_hand_15,z_right_hand_16,z_right_hand_17,z_right_hand_18,z_right_hand_19,z_right_hand_20,sequence_id
str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
"""count""",156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,…,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0,156162.0
"""null_count""",0.0,1411.0,1411.0,1411.0,1411.0,1411.0,1411.0,1411.0,1411.0,1411.0,1411.0,1411.0,1411.0,1411.0,1411.0,1411.0,1411.0,1411.0,1411.0,1411.0,1411.0,1411.0,1411.0,1411.0,1411.0,1411.0,1411.0,1411.0,1411.0,1411.0,1411.0,1411.0,1411.0,1411.0,1411.0,1411.0,…,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,80653.0,80653.0,80653.0,80653.0,80653.0,80653.0,80653.0,80653.0,80653.0,80653.0,80653.0,80653.0,80653.0,80653.0,80653.0,80653.0,80653.0,80653.0,80653.0,80653.0,80653.0,0.0
"""mean""",101.093083,0.563013,0.557142,0.560185,0.545311,0.556668,0.557428,0.560354,0.455933,0.560833,0.560458,0.560851,0.563271,0.5637,0.564292,0.56457,0.564707,0.564898,0.565339,0.564664,0.558318,0.545475,0.406113,0.498286,0.482992,0.468494,0.450011,0.511321,0.474611,0.492097,0.4586,0.448552,0.438191,0.507796,0.450879,0.400089,…,-2.785671,-1.453414,-2.770282,-1.304921,-2.54531,-0.143517,0.149564,-0.054814,0.447185,1.080241,1.492772,1.133415,1.575108,0.11851,0.501641,4.8753e-7,-0.036546,-0.061576,-0.087806,-0.112605,-0.038711,-0.086868,-0.118698,-0.135933,-0.048647,-0.10291,-0.129127,-0.138171,-0.064245,-0.122299,-0.133012,-0.127741,-0.082538,-0.126594,-0.130865,-0.125196,2.7496e8
"""std""",78.886128,0.126063,0.123258,0.124706,0.12464,0.123161,0.12373,0.125801,0.131057,0.126655,0.126762,0.128048,0.126075,0.126194,0.126377,0.126478,0.126401,0.126394,0.126501,0.126437,0.123754,0.124709,0.136842,0.129305,0.12983,0.130463,0.131583,0.128885,0.129689,0.129122,0.130405,0.131031,0.132411,0.128859,0.131402,0.137222,…,1.093977,0.739646,1.040531,0.715696,0.989079,0.102623,0.103301,0.294104,0.459472,0.478034,0.783589,0.496507,0.821186,0.483074,0.759879,6.2225e-7,0.024056,0.03716,0.048194,0.060688,0.044187,0.057319,0.065605,0.071907,0.044602,0.060212,0.066976,0.073256,0.045484,0.059847,0.064845,0.071802,0.048081,0.059104,0.0643,0.070396,8.5118e6
"""min""",0.0,0.143113,0.146648,0.145658,0.125014,0.147784,0.145696,0.137697,-0.014709,0.133423,0.131958,0.125461,0.143131,0.142541,0.141922,0.141972,0.142445,0.142506,0.142089,0.138444,0.147753,0.127774,-0.098558,0.046778,0.027056,0.006738,-0.02384,0.062819,0.016463,0.03861,-0.005472,-0.020369,-0.038455,0.065767,-0.02314,-0.09

## Dataloader

## Preprocessing

### Capsule nets and CNN's?  
The state of the art in face expression recognition is CNN's but capsnet is also suggested to have great performance.

Things to train:
- CNN
- Capsnet inspired by bi-capsnet and 
- GAN


## Training DNN

## Model evaluator: